In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
data = {
    "Age" : 29,
    "Movie" : ["The ShawShank Redemption", "Fight Club"],
    "Movie Ratings" : [9.0, 9.7],
    "Suggestion" : "Inception",
    "Suggestion Purchased" : 1.0,
    "Purchase Price" : 9.99
}
print(data)

{'Age': 29, 'Movie': ['The ShawShank Redemption', 'Fight Club'], 'Movie Ratings': [9.0, 9.7], 'Suggestion': 'Inception', 'Suggestion Purchased': 1.0, 'Purchase Price': 9.99}


In [7]:
example = tf.train.Example(
    features = tf.train.Features(
        feature = {
            "Age" : tf.train.Feature(
                int64_list = tf.train.Int64List(value = [data["Age"]])   
            ),
            "Movie" : tf.train.Feature(
                bytes_list = tf.train.BytesList(value = [m.encode("utf-8") for m in data["Movie"]] )
            ),
            "Movie Ratings" : tf.train.Feature(
                float_list = tf.train.FloatList(value = [i for i in data["Movie Ratings"]])
            ),
            "Suggestion" : tf.train.Feature(
                bytes_list = tf.train.BytesList(value = [data["Suggestion"].encode("utf-8")])
            ),
            "Suggestion Purchased" : tf.train.Feature(
              float_list = tf.train.FloatList(value = [data["Suggestion Purchased"]])  
            ),
            "Purchase Price" : tf.train.Feature(
                float_list = tf.train.FloatList(value = [data["Purchase Price"]])
            )
        }
    )
)

print(example)

features {
  feature {
    key: "Age"
    value {
      int64_list {
        value: 29
      }
    }
  }
  feature {
    key: "Movie"
    value {
      bytes_list {
        value: "The ShawShank Redemption"
        value: "Fight Club"
      }
    }
  }
  feature {
    key: "Movie Ratings"
    value {
      float_list {
        value: 9.0
        value: 9.699999809265137
      }
    }
  }
  feature {
    key: "Purchase Price"
    value {
      float_list {
        value: 9.989999771118164
      }
    }
  }
  feature {
    key: "Suggestion"
    value {
      bytes_list {
        value: "Inception"
      }
    }
  }
  feature {
    key: "Suggestion Purchased"
    value {
      float_list {
        value: 1.0
      }
    }
  }
}



In [8]:
with tf.python_io.TFRecordWriter("customer_1.tfrecord") as writer:
    writer.write(example.SerializeToString())

In [19]:
try:
    sess.close()
except Exception:
    pass
sess = tf.InteractiveSession()


reader = tf.TFRecordReader()

filename_queue = tf.train.string_input_producer(["customer_1.tfrecord"])

key, serilized_example = reader.read(filename_queue)

read_features = {
    "Age" : tf.FixedLenFeature([], dtype = tf.int64),
    "Movie" : tf.VarLenFeature(dtype = tf.string),
    "Movie Ratings" : tf.VarLenFeature(dtype = tf.float32),
    "Suggestion" : tf.FixedLenFeature([], dtype = tf.string),
    "Suggestion Purchased" : tf.FixedLenFeature([], dtype = tf.float32),
    "Purchase Price" : tf.FixedLenFeature([], dtype = tf.float32),
}


read_data = tf.parse_single_example(serialized = serilized_example, features = read_features)

tf.train.start_queue_runners(sess)

for name, tensor in read_data.items():
    print("{} : {}".format(name, tensor.eval()))





Movie : SparseTensorValue(indices=array([[0],
       [1]], dtype=int64), values=array([b'The ShawShank Redemption', b'Fight Club'], dtype=object), dense_shape=array([2], dtype=int64))
Movie Ratings : SparseTensorValue(indices=array([[0],
       [1]], dtype=int64), values=array([9. , 9.7], dtype=float32), dense_shape=array([2], dtype=int64))
Age : 29
Purchase Price : 9.989999771118164
Suggestion : b'Inception'
Suggestion Purchased : 1.0
